In [30]:
import geopandas
from shapely import Polygon


import os
import json
import wget
import zipfile

from dotenv import load_dotenv

from openai import OpenAI

In [4]:

def download_national_parks_data(
        url_path: str = "https://opendata.arcgis.com/datasets/beb04b0e5bed44da9e5407ea50477a5c_0.zip",
        output_directory_base: str = "../data/parques_nacionales",
        raw_data: str = '../data/raw',
        name: str = "data_parques"
) -> str:
    """
    Download the national parks data from the url_path and save it in the output_directory_base
    """
    if not os.path.exists(output_directory_base):
        os.makedirs(output_directory_base, exist_ok=True)

    if not os.path.exists(raw_data):
        os.makedirs(raw_data, exist_ok=True)

    # Download the data
    output_directory = wget.download(url_path, out=raw_data)
    # Unzip the data
    with zipfile.ZipFile(output_directory, 'r') as zip_ref:
        zip_ref.extractall(output_directory_base)

    for file in os.listdir(output_directory_base):
        os.rename(
            os.path.join(output_directory_base, file), 
            os.path.join(output_directory_base, name + os.path.splitext(file)[1])
        )
    return output_directory_base

In [17]:
load_dotenv("../.env")

name = "data_parques"
output_directory_base = download_national_parks_data(name=name)

# Load files
df = geopandas.read_file(
    os.path.join(output_directory_base, name + ".shp")
)

In [18]:
df_demo = geopandas.read_file(
    "../data/demo_data/doc.kml"
)

In [19]:
df_kms = df.to_crs("EPSG:32633")
demo_geometry_kms = df_demo.to_crs("EPSG:32633")

In [31]:
class NationalParkAssistant:

    def __init__(self, df: geopandas.GeoDataFrame, client: OpenAI):
        self.df = df
        self.client = client

    def system_prompt(self) -> str:
        
        
        system_prompt = """
        Eres un experto en parques nacionales y te han contratado para hacer la evaluación de impacto ambiental
        de un nuevo proyecto fotovoltaico en Chile.

        El usuario entregará su locacion y los parques nacionales cercanos en un radio de 3km.

        Tu trabajo es entregar una detallada evaluación de impacto ambiental del proyecto fotovoltaico en los parques nacionales cercanos.

        Para esto primero vas a entregar un resumen si encuentras algo critico, para rechazar el proyecto, si no di que todo esta bien.

        En caso de que encuentres algo critico, debes entregar una evaluación detallada de los impactos ambientales y sociales del proyecto.

        El output será un JSON con dos campos
        ```json
        {
            "resumen": "El proyecto fotovoltaico tiene un impacto ambiental critico en el parque nacional Torres del Paine",
            "evaluacion": "El proyecto fotovoltaico tiene un impacto ambiental critico en el parque nacional Torres del Paine debido a la alta concentración de aves en la zona"}
        }
        ```
        """
        return system_prompt
    
    def format_message(self, close_parks: geopandas.GeoDataFrame) -> str:
        close_parks_data = close_parks.to_json()
        message = """
        Los parques nacionales cercanos a menos de 3km son:
        {close_parks}
        """
        return message.replace("{close_parks}", close_parks_data)


    def evaluate_project(self, location: Polygon, threshold_in_meters: int = 3000) -> dict:
        distance = self.df.geometry.distance(location.iloc[0].geometry)

        close_parks = self.df[distance < 3000]
        
        system_prompt = self.system_prompt()
        message = self.format_message(close_parks)

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": message}
        ]
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            response_format={ "type": "json_object" },
            temperature=0
        )
        return json.loads(response.choices[0].message.content)

In [32]:
park_assistant = NationalParkAssistant(df_kms, OpenAI())

In [33]:
response = park_assistant.evaluate_project(demo_geometry_kms)

In [35]:
response

{'resumen': 'Todo parece estar bien, no se encontraron parques nacionales cercanos al proyecto fotovoltaico.',
 'evaluacion': 'Ninguna evaluación detallada necesaria.'}